In [ ]:
import pickle
import pandas as pd

with open('fertilizer_model_pipeline.pkl', 'rb') as f:
    saved_objects = pickle.load(f)

best_rf = saved_objects['model']
pt = saved_objects['scaler']
le_crop = saved_objects['le_crop']
le_soil = saved_objects['le_soil']
le_fert = saved_objects['le_fert']


In [ ]:
def recommend_fertilizer(input_data):

    df_input = pd.DataFrame([input_data])

    df_input['Crop_le'] = le_crop.transform(df_input['Crop'])
    df_input['Soil_le'] = le_soil.transform(df_input['Soil'])

    numeric_cols = ['Nitrogen', 'Phosphorous', 'Potassium', 'PH', 'Temperature',
                    'Moisture', 'Rainfall', 'Carbon']

    df_input[numeric_cols] = pt.transform(df_input[numeric_cols])

    features = numeric_cols + ['Crop_le', 'Soil_le']
    X_input = df_input[features]

    pred_encoded = best_rf.predict(X_input)[0]

    pred_fertilizer = le_fert.inverse_transform([pred_encoded])[0]

    return pred_fertilizer


In [ ]:
# Example input dictionary
sample_input = {
    'Nitrogen': 54.14082354, 
    'Phosphorous': 134.7425253,
    'Potassium': 159.5640307,
    'PH': 3.908612908,
    'Temperature': 28.9891225, # in celcius
    'Moisture': 0.460655459, # soil moisture, standard soil moisture unit
    'Rainfall': 32.87016218, #in mm
    'Carbon': 0.800157046, #in %-age
    'Crop': 'Mung Bean', 
    'Soil': 'Neutral Soil' #dropdown-> Acidic Soil , Peaty Soil , Neutral Soil  , Loamy Soil  ,Alkaline Soil 
}
#result:  Lime
recommendation = recommend_fertilizer(sample_input)
print("Recommended Fertilizer:", recommendation)

#crop dropdown -> rice, wheat, Mung Bean,  Tea , millet, maize ,Lentil, Jute,  Coffee, Cotton, Ground Nut, Peas ,Rubber ,Sugarcane ,Tobacco ,
# KidneyBeans, Moth Beans 
#Coconut , Black gram ,Adzuki, Beans ,Pigeon Peas, Chickpea, banana, grapes, apple, mango ,muskmelon, orange, papaya, pomegranate, watermelon 

Recommended Fertilizer: Lime
